# Kurvanpassning med Python - _för hand_

Först ska vi ladda lite externa bibliotek

In [ ]:
from matplotlib import pyplot as plt
import scipy.optimize as opt
import numpy as np
import math
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 9]

Sedan ange vi vår mätdata i källkoden i form av två listor: 

uppmätta spänningar $U_{meas}$ `Umeas` och uppmätta strömar $I_{meas}$ `Imeas`

In [ ]:
Umeas = [ 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, -1.0, -2.0, -3.0, -4.0, -5.0, -6.0, -7.0, -8.0, -9.0, -10.0]
Imeas = [ 0.0, 2.8, 5.8, 8.8, 11.7, 14.7, 17.7, 20.7, 23.7, 26.6, 29.6, -2.8, -5.8, -8.7, -11.7, -14.7, -17.7, -20.6, -23.6, -26.6, -29.6]

Och sedan ritar vi upp våra mätpunkter med matplotlib.pyplot.plot()

In [ ]:
fig, ax = plt.subplots() 
ax.plot(Umeas, Imeas)

Vi kan anpassa utseendet av kurvan med olika parameter/attribut

In [ ]:
fig, ax = plt.subplots()
ax.plot(
    Umeas, Imeas, 
    color="blue", 
    linestyle=":", 
    linewidth=2, 
    marker="o", 
    markeredgecolor="blue", 
    markerfacecolor="red", 
    markersize=15
)

In [ ]:
fig, ax = plt.subplots()
ax.set(xlim=[-2,2], ylim=[-10,10])
ax.plot(
    Umeas, Imeas, 
    color="blue", 
    linestyle=":", 
    linewidth=2, 
    marker="o", 
    markeredgecolor="blue", 
    markerfacecolor="red", 
    markersize=15
)

Varifrån kommer den dubbla linjen?

Våra mätpunkter är osorterade - först kommer värden med positiva spänningar, sedan hoppar listan från den högsta positiva spänningen till den första negativa punkten.

Och Python bara drar en linje från punkt till punkt här.

Det är därför bättre att sortera mätpunkterna - inte för hand, fast med Python.

In [ ]:
data = list( zip( Umeas, Imeas ) )
sortdata = sorted(data)

x = [row[0] for row in sortdata]
y = [row[1] for row in sortdata]

print(sortdata)

In [ ]:
print(x)

In [ ]:
print(y)

I Matlab/Octave finns det en kortform som konfigurerar hur en kurva ska ritas. Färg, symbolerna och linjetypen anges i form av en textsträng. Pythons bibliotek Matplotlib stödjer exakt samma funtion. 

`"b+:"` betyder att kurvan ska ritas med kryss som symbol, i blå färg och med en punkterad linje.

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y, "b+:")

Mätpunkterna kommer från ett experiment med en spänningskälla, ett motstånd och en amperemeter. 

Därför borde Ohms lag gäller $I = U/R$ och för att hantera systematiska fel lägger vi till en konstant $I_{offs}$.

Låt oss se hur kurvan skulle se ut för $R = 300\,\text{Ω}$.

In [ ]:
def Iteor(U, R, Ioffs):
    return U / R * 1000 + Ioffs

calc_x = np.arange(-12, 12, 0.01)
calc_y = Iteor(calc_x, 300, 0)

`calc_x = np.arange(-12, 12, 0.01)` skapar en lista med spänningsvärden mellan $-12\,\text{V}$ och $+12\,\text{V}$ med steg på $0.01\,\text{V}=10\,\text{mV}$

`calc_y = Iteor(calc_x, 300, 0)` skapar en lista med ett funktionsvärde $f(x)=mx+b$ för alla värden från listan `calc_x`

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y, color="red", linestyle=":", linewidth=2, marker="o",
         markeredgecolor="blue", markerfacecolor="red", markersize=15)
ax.plot(calc_x, calc_y, "g-")

Lutningen verkar inte stämma helt. Den gröna kurvan från den teoretiska modellen är brantare än våra mätpunkter. Gå tillbaka och ändra värdet 300 i `calc_y = Iteor(calc_x, 300, 0)` som representerar resistansen $R$ i ohm.

## Kurvanpassning

I föreläsningen presenterades en analytisk lösning för en linjär kurvanpassning, dvs för uppgiften att hitta en modellfunktion $f(x)=mx + b$ som minimerar summan av alla kvadratiska avvikelser mellan mätvärden $y_i$ och de framräknade värden från modellfunktionen:

$\displaystyle S(m,b) = \sum_i \left(y_i - f(x_i)\right)^2 = \sum_i \left(y_i - m x_i - b\right)^2$

De partiella derivatorna blir noll vid extrempunkterna av $S$ och ger oss sedan ett lösbart ekvationssystem för $m$ och $b$

$\displaystyle\frac{\partial S}{\partial m} = -2\sum_{i=1}^{n}x_i y_i  +2m\sum_{i=1}^{n} x_i^2 + 2b\sum_{i=1}^{n}x_i  =  -2\alpha + 2m\beta +2b\gamma = 0$

och

$\displaystyle\frac{\partial S}{\partial b} = -2\sum_{i=1}^{n}y_i  +2m\sum_{i=1}^{n} x_i + 2 b \sum_{i=1}^{n}1 = -2\delta + 2m\gamma + 2 b n = 0$

med $\displaystyle\alpha = \sum_{i=1}^{n}x_i y_i\qquad$
$\displaystyle\beta = \sum_{i=1}^{n} x_i^2\qquad$
$\displaystyle\gamma = \sum_{i=1}^{n}x_i\qquad$
$\displaystyle\delta = \sum_{i=1}^{n}y_i$

$\displaystyle m = \frac{\alpha - \frac{\gamma\delta}{n}}{\beta-\frac{\gamma^2}{n}}$

$\displaystyle b = \frac{\delta - m\gamma}{n}$

Summorna kan vi enkelt låta Python räkna ut:

In [ ]:
alpha = 0 # = sum x_i y_i
beta  = 0 # = sum x_i^2
gamma = 0 # = sum x_i
delta = 0 # = sum y_i
n = len(x)

In [ ]:
for i in range(len(x)):
    alpha = alpha + x[i]*y[i]
    beta  = beta + x[i]*x[i]
    gamma = gamma + x[i]
    delta = delta + y[i]

Och så kan vi räkna ut $m$ och $b$

In [ ]:
m = (alpha - gamma*delta/n)/(beta-gamma*gamma/n)

b = (delta - gamma * m)/n

print("f(x) = {:+f}x {:+f}".format(m,b))

Och med dessa optimala $m$ och $b$ räkna vi på nytt ut den teoretiska kurvan från vår modellfunktion och rita upp tillsammans med våra mätpunkter

In [ ]:
calc_y = Iteor(calc_x, 1000/m, b)

fig, ax = plt.subplots()
ax.plot(x, y, color="red", linestyle=":", linewidth=2, marker="o",
         markeredgecolor="blue", markerfacecolor="red", markersize=15)
ax.plot(calc_x, calc_y, "b-")